## 4.4 勾配

- $x_0$と$x_1$の偏微分をまとめて計算する
- たとえば $x_0=3,x_1=4$のときの$(x_0, x_1)$の両方の偏微分をまとめて、
$\left(\frac{\partial f}{\partial x_0},\frac{\partial f}{\partial x_1}\right)$として計算する

- $\left(\frac{\partial f}{\partial x_0},\frac{\partial f}{\partial x_1}\right)$のように、すべての変数の偏微分をベクトルとしてまとめたものを勾配(gradient)という。

In [1]:
import numpy as np

In [2]:
# 勾配の実装例
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x) # xと同じ形状の配列を生成
    
    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x+h)の計算
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h)の計算
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 値を元に戻す
        
    return grad
        

In [3]:
# 実装
def function_2(x):
    return x[0]**2 + x[1]**2
    # または
    # return np.sum(x**2)

##### function_2のグラフ

In [4]:
from mpl_toolkits.mplot3d import Axes3D

In [5]:
x0 = np.arange(-3, 3, 0.1)
x1 = np.arange(-3, 3, 0.1)
X0, X1 = np.meshgrid(x0, x1)
Y = X0 ** 2 + X1 ** 2

fig = plt.figure()
ax = Axes3D(fig)
ax.plot_wireframe(X0,X1,Y)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
numerical_gradient(function_2,np.array([3.0,4.0]))

In [ ]:
numerical_gradient(function_2,np.array([0.0,2.0]))

In [ ]:
numerical_gradient(function_2,np.array([3.0,0.0]))

#### gradient_2d.py

In [ ]:
# coding: utf-8
# cf.http://d.hatena.ne.jp/white_wheels/20100327/p3
import numpy as np
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D


def _numerical_gradient_no_batch(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        
    return grad


def numerical_gradient(f, X):
    if X.ndim == 1:
        return _numerical_gradient_no_batch(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_no_batch(f, x)
        
        return grad


def function_2(x):
    if x.ndim == 1:
        return np.sum(x**2)
    else:
        return np.sum(x**2, axis=1)


def tangent_line(f, x):
    d = numerical_gradient(f, x)
    print(d)
    y = f(x) - d*x
    return lambda t: d*t + y
     
if __name__ == '__main__':
    x0 = np.arange(-2, 2.5, 0.25)
    x1 = np.arange(-2, 2.5, 0.25)
    X, Y = np.meshgrid(x0, x1)
    
    X = X.flatten()
    Y = Y.flatten()
    
    grad = numerical_gradient(function_2, np.array([X, Y]) )
    
    plt.figure()
    plt.quiver(X, Y, -grad[0], -grad[1],  angles="xy",color="#666666")#,headwidth=10,scale=40,color="#444444")
    plt.xlim([-2, 2])
    plt.ylim([-2, 2])
    plt.xlabel('x0')
    plt.ylabel('x1')
    plt.grid()
    plt.legend()
    plt.draw()
    plt.show()

### 4.4.1 勾配法

- 機械学習では最適なパラメータを探索する(学習することで見つけ出す)
- ニューラルネットワークでは重みとバイアス(の最適値)を探す
- 勾配を利用して、関数の最小値(最小というよりは、できるだけ小さな値=極小値)
- 勾配の指す先に進めば、より小さな値に近づけるはずだが、それが関数全体の最小値とは限らない
- なぜならば
  - デコボコした複雑な関数の場合、そこが最小値とは限らない
  - 最大値かもしれない(見る方向によっては、最小値=最大値)
- すなわち見つけ出せるのは「鞍点」に過ぎない
  - 関数が複雑なとき「プラトー」に陥ることがある(学習が進まない状態)
- それでもまあ、この方法なら、値を減らせることは間違いないので、まずはこれで良しとする、という方法。



- 数式で表す

$$
x_0 = x_0 - \eta\frac{\partial f}{\partial x_0}
$$
$$
x_1 = x_1 - \eta\frac{\partial f}{\partial x_1}
$$
- ここで$ \eta\ $は更新量
   - ニューラルネットワークの学習においては**学習率**と呼ばれる
   - １回の学習で、どれだけ学習すべきか、どれだけパラメータを更新するか、ということを決める
   



#### 勾配降下法のPython実装


In [ ]:
# fは最適化したい関数
# init_xは初期値
# lr はlearning rate 学習率
# step_numは勾配法による繰り返しの数
# この関数で求められるのは「極小値」（うまく行けば最小値）
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

##### 問: $ f(x_0) = x_0^2 + x_1^2 $の最小値を勾配法で求めよ。


In [ ]:
def function_2(x):
    return x[0]**2 + x[1]**2

In [ ]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100)

###### 勾配法による更新のプロセスの図示化

In [ ]:
# coding: utf-8
import numpy as np
import matplotlib.pylab as plt
# from gradient_2d import numerical_gradient


def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    x_history = []

    for i in range(step_num):
        x_history.append( x.copy() )

        grad = numerical_gradient(f, x)
        x -= lr * grad

    return x, np.array(x_history)


def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])    

lr = 0.1
step_num = 20
x, x_history = gradient_descent(function_2, init_x, lr=lr, step_num=step_num)

plt.plot( [-5, 5], [0,0], '--b')
plt.plot( [0,0], [-5, 5], '--b')
plt.plot(x_history[:,0], x_history[:,1], 'o')

plt.xlim(-3.5, 3.5)
plt.ylim(-4.5, 4.5)
plt.xlabel("X0")
plt.ylabel("X1")
plt.show()


#### 学習率の違いによる結果の実験

In [ ]:
# fは最適化したい関数
# init_xは初期値
# lr はlearning rate 学習率
# step_numは勾配法による繰り返しの数
# この関数で求められるのは「極小値」（うまく行けば最小値）
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

In [ ]:
# 学習率が大きすぎる例：lr=10.0
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100)

In [ ]:
# 学習率が小さすぎる例：lr = 1e-10
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=1e-10, step_num=100)

#### 学習率の違いによる結果の実験の可視化

In [ ]:
def gradient_descent_history(f, init_x, lr=0.01, step_num=100):
    x = init_x
    x_history = []

    for i in range(step_num):
        x_history.append( x.copy() )

        grad = numerical_gradient(f, x)
        x -= lr * grad

    return x, np.array(x_history)

In [ ]:
# 学習率が大きすぎる例：lr=10.0
init_x = np.array([-3.0, 4.0])    
lr = 10.0
step_num = 100
x, x_history = gradient_descent_history(function_2, init_x, lr=lr, step_num=step_num)

plt.plot( [-5, 5], [0,0], '--b')
plt.plot( [0,0], [-5, 5], '--b')
plt.plot(x_history[:,0], x_history[:,1], 'o')

plt.xlim(-3e+13, 3e+13) ## 変えた
plt.ylim(-2e+12, 2e+12) ## 変えた
plt.xlabel("X0")
plt.ylabel("X1")
plt.show()

In [ ]:
# 学習率が小さすぎる例：lr = 1e-10
init_x = np.array([-3.0, 4.0])    
lr = 1e-10
step_num = 100
x, x_history = gradient_descent_history(function_2, init_x, lr=lr, step_num=step_num)

plt.plot( [-5, 5], [0,0], '--b')
plt.plot( [0,0], [-5, 5], '--b')
plt.plot(x_history[:,0], x_history[:,1], 'o')

plt.xlim(-3.5, 3.5)
plt.ylim(-4.5, 4.5)
plt.xlabel("X0")
plt.ylabel("X1")
plt.show()

### 4.4.2 ニューラルネットワークに対する勾配

- たとえば、形状が２ｘ３で、重みＷだけを持つニューラルネットワークにおいて損失関数をＬで表す場合を考える
- 勾配は$ \frac{\partial L}{\partial W} $

$$
W = \left(
        \begin{array}{ccc}
            w_{11} & w_{12} & w_{13} \\
            w_{21} & w_{22} & w_{23} \\
        \end{array}
    \right)
$$

$$
\frac{\partial L}{\partial W} = 
    \left(
        \begin{array}{ccc}
            \frac{\partial L}{\partial w_{11}} & \frac{\partial L}{\partial w_{12}} & \frac{\partial L}{\partial w_{13}} \\
            \frac{\partial L}{\partial w_{21}} & \frac{\partial L}{\partial w_{22}} & \frac{\partial L}{\partial w_{23}} \\
        \end{array}
    \right)
$$

### $ 2 \times 3 $ の簡単なニューラルネットワークsimpleNetクラスの実装

In [23]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # ガウス分布で初期化
        
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y,t)
        
        return loss

In [24]:
net = simpleNet()
print(net.W) # 重みパラメータ

[[ 0.47960288 -0.76193959  1.8855813 ]
 [-2.3624579  -0.50910295  0.16812907]]


In [25]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[-1.83845038 -0.9153564   1.28266494]


In [26]:
np.argmax(p) # 最大値のインデックス

2

In [27]:
t = np.array([0,0,1]) # 正解ラベル
net.loss(x, t)

0.14421326360761949

### 勾配を求めてみる

In [29]:
def f(W):
    return net.loss(x, t)

In [30]:
dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.02291063  0.05766756 -0.08057819]
 [ 0.03436594  0.08650134 -0.12086728]]


##### lambda記法

In [31]:
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)

In [32]:
print(dW)

[[ 0.02291063  0.05766756 -0.08057819]
 [ 0.03436594  0.08650134 -0.12086728]]


## 4.5 学習アルゴリズムの実装

### 4.5.1 2層ニューラルネットワークのクラス

2層のニューラルネットワーク（隠れ層が1層のニューラルネットワーク）を対象に、MNISTデータセットを使って学習を行う

In [44]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import *
from common.gradient import numerical_gradient

In [61]:
class TwoLayerNet:
    # このクラスで使用する変数
    ## params : ニューラルネットワークのパラメータを保持するディクショナリ変数（インスタンス変数）
    ##          params['W1']は1層目の重み、params['b1']は1層目のバイアス。
    ##          params['W2']は2層目の重み、params['b2']は2層目のバイアス。
    #
    ## grads : 勾配を保持するディクショナリ変数（numerical_gradient()メソッドの返り値）
    ##         grads['W1']は1層目の重みの勾配、grads['b1']は1層目のバイアスの勾配。
    ##         grads['W2']は2層目の重みの勾配、grads['b2']は2層目のバイアスの勾配。
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y,t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    

#### 一つ例を見てみる

In [62]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

In [63]:
net.params['W1'].shape # 今の層（入力層）のニューロン数×次の層（隠れ層）ニューロン数の行列になる

(784, 100)

In [64]:
net.params['b1'].shape # 次の層（隠れ層）のニューロン数の行列になる

(100,)

In [65]:
net.params['W2'].shape # 今の層（隠れ層）のニューロン数×次の層（出力層）のニューロン数の行列になる

(100, 10)

In [66]:
net.params['b2'].shape # 次の層（出力層）のニューロン数の行列になる

(10,)

#### 推論処理の例